In [25]:
import pandas as pd
from bokeh.models import Slider, CustomJS, ColumnDataSource, NumeralTickFormatter, Legend, LegendItem
from bokeh.layouts import column, row
from bokeh.palettes import Category10
from bokeh.plotting import figure

def create_migration_map(df):
    # Prepare yearly data by district
    yearly_data = df.groupby(['Year', 'PdDistrict']).size().unstack(fill_value=0)
    
    # Get top districts and normalize
    top_districts = yearly_data.sum().nlargest(8).index.tolist()
    yearly_data = yearly_data[top_districts]
    
    # Normalize by yearly totals to show relative changes
    yearly_data = yearly_data.div(yearly_data.sum(axis=1), axis=0)
    
    # Create sources
    source = ColumnDataSource(yearly_data.reset_index())
    
    # Create figure with adjusted width
    p = figure(
        title="Crime Hotspot Migration (Normalized by Year)",
        x_range=(2003, 2025),
        y_range=(0, yearly_data.max().max()*1.1),
        width=800,
        height=500,
        tools="hover,pan,box_zoom,reset",
        tooltips=[("Year", "@Year"), ("District", "$name"), ("Percentage", "@$name{0.0%}")],
        toolbar_location="above"
    )
    
    # Add lines for each district with Category10 palette
    colors = Category10[10][:len(top_districts)]
    renderers = []
    legend_items = []
    
    # Districts to show initially (others will be muted)
    districts_to_show = []
    
    for i, district in enumerate(top_districts):
        # Set muted=True for districts not in our show list
        muted_initially = district not in districts_to_show
        
        r = p.line(
            x='Year',
            y=district,
            source=source,
            line_width=3,
            color=colors[i],
            alpha=0.8 if not muted_initially else 0.2,
            muted_alpha=0.1,
            muted=muted_initially,
            name=district
        )
        renderers.append(r)
        legend_items.append(LegendItem(label=district, renderers=[r], index=i))  # Added index for color reference
    
    # Create a dummy plot to hold the legend
    dummy = figure(width=200, height=p.height, toolbar_location=None,
                  outline_line_color=None, min_border=0)
    dummy.axis.visible = False
    dummy.grid.visible = False
    
    # Create legend with proper color indicators
    legend = Legend(
        items=legend_items,
        click_policy="mute",
        location="center",
        label_text_font_size="10pt",
        glyph_width=20,  # Size of the color indicator
        glyph_height=20,
        spacing=10,
        margin=10,
        padding=5
    )
    dummy.add_layout(legend)
    
    # Add interactive slider
    slider = Slider(
        start=2003,
        end=2024,
        value=2014,
        step=1,
        title="Highlight Year",
        width=800  # Match plot width
    )
    
    # Add vertical marker for slider
    vline = p.line(
        x=[2014, 2014],
        y=[0, 1],
        line_width=2,
        line_dash='dashed',
        color='red'
    )
    
    # JavaScript callback for slider
    slider.js_on_change('value', CustomJS(
        args=dict(vline=vline, source=source),
        code="""
        vline.data_source.data['x'] = [cb_obj.value, cb_obj.value]
        vline.data_source.change.emit()
        """
    ))
    
    # Styling
    p.yaxis.formatter = NumeralTickFormatter(format="0%")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    # Create layout
    plot_row = row(dummy, p)
    full_layout = column(slider, plot_row)
    
    return full_layout

# Usage
migration_plot = create_migration_map(df)
show(migration_plot)

In [20]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, NumeralTickFormatter, Legend, LegendItem
from bokeh.palettes import Category10
import pandas as pd

def create_hourly_crime_comparison(df):
    # Prepare data - get hourly counts for all crimes
    hourly_counts = df.groupby(['Category', 'TimeOfDay']).size().unstack(fill_value=0)
    normalized = hourly_counts.div(hourly_counts.sum(axis=1), axis=0)
    
    # Convert to long format for Bokeh
    normalized = normalized.reset_index().melt(id_vars='Category', 
                                            var_name='Hour', 
                                            value_name='Frequency')
    
    # Create mapping of crime types to colors
    crimes = normalized['Category'].unique().tolist()
    color_map = {crime: Category10[10][i % 10] for i, crime in enumerate(crimes)}
    
    # Create figure with wider dimensions
    p = figure(
        x_range=[str(i) for i in range(24)],
        title="Hourly San Francisco Crime Patterns",
        width=1200,
        height=600,
        tools="hover,pan,box_zoom,reset",
        tooltips="@Crime: @Frequency{0.00%} at @Hour:00"
    )
    
    # Add lines for each crime type
    renderers = []
    legend_items = []
    
    # Define which crimes should start muted (all except these will be muted)
    crimes_to_show = []
    
    for crime in crimes:
        crime_data = normalized[normalized['Category'] == crime]
        source = ColumnDataSource({
            'Hour': crime_data['Hour'].astype(str),
            'Frequency': crime_data['Frequency'],
            'Crime': [crime] * len(crime_data)
        })
        
        # Set muted=True for crimes not in our show list
        muted_initially = crime not in crimes_to_show
        
        r = p.line(
            x='Hour',
            y='Frequency',
            source=source,
            line_width=2,
            color=color_map[crime],
            alpha=0.6,
            muted_alpha=0.1,
            muted=muted_initially,  # This controls initial mute state
            name=crime
        )
        renderers.append(r)
        legend_items.append(LegendItem(label=crime, renderers=[r]))
    
    # Create separate legend and add it to the plot's left side
    legend = Legend(items=legend_items, click_policy="mute", location="center")
    p.add_layout(legend, 'left')
    
    # Formatting
    p.xaxis.axis_label = "Hour of Day"
    p.yaxis.axis_label = "Normalized Crime Frequency"
    p.yaxis.formatter = NumeralTickFormatter(format="0%")
    p.xgrid.grid_line_color = None
    
    # Adjust plot margins to make room for the legend
    p.min_border_left = 250
    
    return p

# Usage
crime_comparison = create_hourly_crime_comparison(df)
show(crime_comparison)